In [4]:
!pip install kagglehub[pandas-datasets]
!pip install pandas
!pip install tensorflow
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [23]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
fake_file_path = "Fake.csv"

# Load the latest version
df_false = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "clmentbisaillon/fake-and-real-news-dataset",
  fake_file_path,
)

true_file_path = "True.csv"

df_true = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "clmentbisaillon/fake-and-real-news-dataset",
  true_file_path,
)

df_true.head()
# df_true.iloc[0]['text']
# df_true.iloc[0]['title']

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [24]:
import warnings
warnings.filterwarnings('ignore')

In [25]:
import pandas as pd

In [26]:
df_false['label'] = 0
df_true['label'] = 1
df = pd.concat([df_false, df_true], ignore_index=True)

In [27]:
df['total_text'] = df['title'] + "\n" + df['text']

In [28]:
print(df.iloc[2]['total_text'])

 Sheriff David Clarke Becomes An Internet Joke For Threatening To Poke People ‘In The Eye’
On Friday, it was revealed that former Milwaukee Sheriff David Clarke, who was being considered for Homeland Security Secretary in Donald Trump s administration, has an email scandal of his own.In January, there was a brief run-in on a plane between Clarke and fellow passenger Dan Black, who he later had detained by the police for no reason whatsoever, except that maybe his feelings were hurt. Clarke messaged the police to stop Black after he deplaned, and now, a search warrant has been executed by the FBI to see the exchanges.Clarke is calling it fake news even though copies of the search warrant are on the Internet. I am UNINTIMIDATED by lib media attempts to smear and discredit me with their FAKE NEWS reports designed to silence me,  the former sheriff tweeted.  I will continue to poke them in the eye with a sharp stick and bitch slap these scum bags til they get it. I have been attacked by be

In [29]:
print(df.shape)
df.drop_duplicates(inplace=True)
print(df.shape)

(44898, 6)
(44689, 6)


In [30]:
from sklearn.model_selection import train_test_split
text = df['total_text']
labels = df['label']
x_train, x_test, y_train, y_test = train_test_split(text, labels, test_size=0.2, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=0.5, random_state=42)

print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)
print(x_test.shape)
print(y_test.shape)

(35751,)
(35751,)
(4469,)
(4469,)
(4469,)
(4469,)


In [34]:
import re
def preprocess_text(text: pd.DataFrame) -> None:
    """
    args:
    
    return:
    """
    return (text.lower()
        .replace('[^a-zA-Z]', ' ')
        .replace('\s+', ' ')
        .replace(r'https?://\S+|www\.\S+|[^a-zA-Z\s]', '')
        .replace(r'<.*?>', ''))

print(x_train.apply(preprocess_text))

7001      trump’s own children won’t be voting for him ...
5219      crowd erupts in boos when military mom asks p...
40156    factbox: malaysia 2018 budget seen lifting cas...
24280    trump wants $4 billion more for missile defens...
42314    turkey will deal with iraqi central government...
                               ...                        
6265      hillary’s message to former miss universe cal...
11285    senior cia intel: “wiretapping of trump transi...
38335    gala glitz masks asia's tensions as trump wind...
860       trump stole an idea from north korean propaga...
15798    watch judge tell disrespectful gangbanger taxp...
Name: total_text, Length: 35751, dtype: object


In [38]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train)
train_seq = tokenizer.texts_to_sequences(x_train)
vocab_size = len(tokenizer.word_index) + 1
max_length = max(len(sequence) for sequence in train_seq)

train_seq = pad_sequences(train_seq, maxlen=max_length, padding='post', truncating='post')

In [43]:
def get_seq(text: pd.DataFrame, tokenizer: Tokenizer, max_length: int) -> pd.DataFrame:
    """
    """
    seq = tokenizer.texts_to_sequences(text)
    return pad_sequences(seq, maxlen=max_length, padding='post', truncating='post')

val_seq = get_seq(x_val, tokenizer, max_length)
test_seq = get_seq(x_test, tokenizer, max_length)

In [46]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Input, Embedding, LSTM, Dropout, Bidirectional
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

epochs = 10
lr = 1e-3
embedding_dim = 300
model = Sequential([
    Input(shape = (max_length,)),
    Embedding(vocab_size, embedding_dim, input_length = max_length, trainable = False),
    
    Bidirectional(LSTM(128, return_sequences = True)),
    Bidirectional(LSTM(64)),
    Dropout(0.2),
    
    Dense(256, activation = 'relu'),

    Dropout(0.5),
    
    Dense(1, activation = 'sigmoid')
])

model.compile(optimizer = Adam(learning_rate = lr), loss = BinaryCrossentropy(), metrics = ['accuracy'])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 8389, 300)      │    40,383,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ (None, 8389, 256)      │       439,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_3 (Bidirectional) │ (None, 128)            │       164,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 41,019,929 (156.48 MB)

 Trainable params: 636,929 (2.43 MB)

 Non-trainable params: 40,383,000 (154.05 MB)

In [47]:
model_es = EarlyStopping(monitor = 'val_loss', mode = 'min', patience = 2, restore_best_weights = True)
model_rlr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2, patience = 1, mode = 'min')

# train the model

history = model.fit(train_seq, y_train, epochs = epochs, 
                    validation_data = (val_seq , y_val), callbacks = [model_es, model_rlr])

Epoch 1/10
   3/1118 ━━━━━━━━━━━━━━━━━━━━ 4:26:40 14s/step - accuracy: 0.5694 - loss: 0.6920

KeyboardInterrupt: 